In [184]:
import pandas as pd
import numpy as np
import time

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 1000)

# Steps
1. empty hand with 1.0 chance
+ or other hands: determine whih step to take next (e.g. plyer finished, split hand etc.)
2. deal random cards with chances
+ combine same hands (5,8 and 8,5)
+ finish blackjacks
3. get all possible moves for player, keep the drawn cards saved
+ vectorized adding (with diag)
+ [FALSE] aggregate bust hands (e.g. a 18 only needs 11,2,3 and 4+ can be immediately seen as a bust hand)
4. get all possible moves for dealer
+ aggregate deck and dealer info as input: same deck info means same dealer possibilities
+ after getting each possible score from deck info, calculate win lose and push ratios for each player hand
+ (same deck info mean same hand? then even better)
5. for each drawn hand starting from the back, determine the best move (hit or stand)
+ save best moves for possible combinations, use them on all possible instead of recaculating
+ dont include obvious cases (e.g. stand on 21, hit under 12)
6. determine expected return for hands as far back as you like (start of game, dealt hands, dealer plays)

In [185]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["score", "cards", "aces"]
ACTIVE_COLUMNS = ["player_"+colname for colname in HAND_COLUMNS] + ["dealer_"+colname for colname in HAND_COLUMNS]
GAME_COLUMNS = ["chance"] + ACTIVE_COLUMNS + DECK_COLUMNS

# SETUP

In [186]:
def get_deck(number_decks=6):
    """get a deck with the specified number of decks (default: 6)"""
    
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [187]:
def order_game_columns(game):
    """order the columns of the game dataframe"""

    return game[GAME_COLUMNS + [colname for colname in game.columns if colname not in GAME_COLUMNS]]

In [188]:
def player_draws_single(game, value, probabilistic=True, update_deck=True):
    """update the game with the player drawing a card with the specified value"""

    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["player_score"] += value
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game["player_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    return next_game

In [189]:
def dealer_draws_single(game, value, probabilistic=True, update_deck=True):
    """update the game with the dealer drawing a card with the specified value"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["dealer_score"] += value
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game["dealer_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -1
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1 

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    return next_game

In [190]:
def get_game(deck=None, player_cards=[], dealer_cards=[], update_deck=True):
    """get a game with the specified deck, player cards and dealer cards"""

    if deck is None:
        deck = get_deck()
    
    game = pd.DataFrame(
        [1.0] + [0, 0, 0]*2 + deck.values.flatten().tolist(), 
        index=GAME_COLUMNS
    ).T.astype(int).astype({"chance":float})

    for player_card_value in player_cards:
        game = player_draws_single(game, player_card_value, probabilistic=False, update_deck=update_deck)
    
    for dealer_card_value in dealer_cards:
        game = dealer_draws_single(game, dealer_card_value, probabilistic=False, update_deck=update_deck)
    
    return game

# PLAYER

In [191]:
def player_draws_once(game, keep_drawn_column=False):
    """get updated game when player hits once, throws error for finished hands"""

    assert np.all(game["dealer_cards"] != 0), f"dealer must have one card showing. hands: {game}"    
    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to hit. hands:\n{game}"
    assert game[DECK_COLUMNS].sum(axis=1).max() == game[DECK_COLUMNS].sum(axis=1).min(), f"there are decks with different number of cards. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["player_score"] += next_game["drawn"]
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "player_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # aggregate same hands
    cols = next_game.columns.tolist()
    cols.remove("chance")
    next_game = next_game.groupby(cols).agg({'chance': 'sum'}).reset_index()
    # NOTE: is drawn column is kept, the hands will not be grouped (useful for player decisions)

    # sort columns
    next_game = order_game_columns(next_game)

    return next_game.sort_values("player_score")


In [192]:
def player_plays_all_possible(game):
    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22) & (game["player_score"] != 21)
    playable = game[is_playable].copy()
    final = game.copy()

    # print("calculating all possible moves...")
    while len(playable) > 0:
        next_game = player_draws_once(playable, keep_drawn_column=True)
        drawnsum = len([i for i in next_game.columns.tolist() if "drawn" in i])
        next_game = next_game.rename(columns={"drawn":"drawn_"+str(drawnsum)})
        final["drawn_"+str(drawnsum)] = np.zeros(len(final)).astype(int)
        
        final = pd.concat((final, next_game), axis=0, ignore_index=True)

        is_playable = (next_game["player_score"] >= 0) & (next_game["player_score"] != 22) & (next_game["player_score"] != 21)
        playable = next_game[is_playable].copy()
    
    return final

# DEALER

In [193]:
def dealer_draws_once(game, keep_drawn_column=False):
    """get updated game when dealer hits once"""
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["dealer_score"] += next_game["drawn"]
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # group and reorder
    cols = next_game.columns.tolist()
    cols.remove("chance")
    next_game = next_game.groupby(cols).agg({'chance': 'sum'}).reset_index()

    return next_game.sort_values("dealer_score")


In [206]:
def dealer_plays(game):
    """get outcomes for dealer play with returns, grouped by original deck info"""
    
    assert set(DECK_COLUMNS).issubset(set(game.columns.tolist())), "Deck Info not in columns"
    assert set(["dealer_score","dealer_aces","dealer_cards"]).issubset(game.columns.tolist()), "not all info in columns"
    assert "initial_samedeck" in game.columns, "initial same deck column not in game"
    assert "drawn" not in game.columns, f"drawn column already exists. hands:\n{game}"

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    while len(playable) > 0:
        next_game = dealer_draws_once(playable)

        is_playable = (next_game["dealer_score"] < 17) & (next_game["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_game[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_game[is_playable].copy()

    final = final.groupby(["dealer_score","initial_samedeck"]).agg({"chance":"sum"}).reset_index()

    return final

In [209]:
def dealer_get_expected_return(game, verbose=False):
    """simulates dealer play, adds column "expected_return" """

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"

    # TODO: get only distinct decks (dealer and deck)
    # get the expected outputs for each distinct
    # merge them back onto the game, calculate returns

    # get distinct decks
    print("game\n", game)
    game["initial_hand"] = np.arange(len(game))
    distinct_decks = game.copy().loc[:, ["dealer_score","dealer_cards","dealer_aces"] + DECK_COLUMNS].drop_duplicates()

    print(f"Simplified from {len(game)} to {len(distinct_decks)} games")
    print("distinct decks\n", distinct_decks)

    distinct_decks["initial_samedeck"] = np.arange(len(distinct_decks)) # for merging back to original game
    game = pd.merge(game, distinct_decks.loc[:, DECK_COLUMNS + ["dealer_score","dealer_cards","dealer_aces"] + ["initial_samedeck"]], on=DECK_COLUMNS + ["dealer_score","dealer_cards","dealer_aces"], how="left") # initial in game
    print("game\n", game)

    distinct_decks["chance"] = 1.0
    dealer_scores_grouped = dealer_plays(distinct_decks)
    dealer_scores_grouped = dealer_scores_grouped.rename(columns={"chance":"normed_chance"})

    print("dealer_scores_grouped\n", dealer_scores_grouped)

    # merge back to original game
    game = game.rename({"dealer_score":"dealer_score_orig","dealer_cards":"dealer_cards_orig","dealer_aces":"dealer_aces_orig"}, axis=1)
    game = pd.merge(game, dealer_scores_grouped.loc[:, ["initial_samedeck","dealer_score","normed_chance"]], on=["initial_samedeck"], how="left")
    game["chance"] = game["normed_chance"] * game["chance"]
    game = get_return_column(game)

    game["expected_return"] = game["chance"] * game["return"]
    game = game.drop(columns=["normed_chance","return","initial_samedeck"])

    game = game.groupby(["initial_hand","player_score","player_cards","player_aces","initial","dealer_score_orig","dealer_cards_orig","dealer_aces_orig"] + [col for col in game.columns.tolist() if "drawn" in col] + DECK_COLUMNS).agg({"expected_return":"sum","chance":"sum"}).reset_index()
    game = game.rename({"dealer_score_orig":"dealer_score","dealer_cards_orig":"dealer_cards","dealer_aces_orig":"dealer_aces"}, axis=1)

    game = game.drop(columns=["initial_hand"])
    print("updated game\n", game)

    return game


# RETURNS

In [196]:
def get_expected_return_standing(game, verbose=False):
    """add column "expected return" to game,
    player has minimum of 2 cards, dealer has 1 upcard
    assumes perfect play"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"

    # bust hands
    is_bust = game["player_score"] == -2
    finished = game[is_bust].copy()
    playable = game[~is_bust].copy()

    if verbose:
        print("Calculating possible returns for each game...")
        print(f"Bust games: {len(finished) / len(game) * 100:.2f}%")

    dealer_outcomes = dealer_get_expected_return(playable, verbose)

    finished["expected_return"] = -finished["chance"]
    
    final = pd.concat((finished, dealer_outcomes), axis=0, ignore_index=True)

    return final


In [197]:
def get_return_column(game):
    """add "return" column to game dataframe, denoting the return of the game"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    game["return"] = pd.Series(np.zeros(len(game)))

    game.loc[(game["player_score"] == 22) & (game["dealer_score"] != 22), "return"] = 1.5
    game.loc[(game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22), "return"] = 1
    game.loc[game["player_score"] == game["dealer_score"], "return"] = 0
    game.loc[game["player_score"] < game["dealer_score"], "return"] = -1

    return game

In [198]:
def get_expected_return_best(game, verbose=False):
    """add column "expected_return" to game dataframe
    assumes best play"""

    assert np.all(game["player_cards"] >= 2), "player has less than 2 cards"
    assert np.all(game["dealer_cards"] == 1), "dealer has no cards showing"

    print("game\n", game.sort_values(["player_score","dealer_score"]))

    t1 = time.time()

    game["initial"] = np.arange(len(game))

    if np.all(game["player_score"] == 22):
        return get_expected_return_standing(game)

    final = player_plays_all_possible(game)
    final = get_expected_return_standing(final, verbose)
    final["stand_return"] = np.nan
    final["hit_return"] = np.nan

    drawn_columns = [i for i in final.columns.tolist() if "drawn" in i]
    
    print(final.sort_values(drawn_columns))

    # get double scores if player has 2 cards and can draw at least once 
    if np.all(game["player_cards"] == 2) and "drawn_1" in drawn_columns and "drawn_2" in drawn_columns:
        if verbose:
            print("double vals\n", final.loc[(final["drawn_1"] != 0) & (final["drawn_2"] == 0), [col for col in final.columns if "drawn" not in col]+["drawn_1"]])
        double_return = final.loc[(final["drawn_1"] != 0) & (final["drawn_2"] == 0), "expected_return"].sum() * 2
    elif np.all(game["player_cards"] == 2) and "drawn_1" in drawn_columns and "drawn_2" not in drawn_columns:
        double_return = final.loc[(final["drawn_1"] != 0), "expected_return"].sum() * 2
    else:
        double_return = None

    for i in range(len(drawn_columns)):
        col = drawn_columns[len(drawn_columns)-i-1]
        cols = drawn_columns[:len(drawn_columns)-i-1]
        print(col)

        # get all games that can stand or hit in that column
        if len(cols) != 0:
            useable = (final[col] != 0) | ((final[col] == 0) & (final[cols[-1]] != 0) & (final["player_score"] > 0) & (final["player_score"] != 21))
        else:
            useable = (final[col] != 0) | ((final[col] == 0) & (final["player_score"] > 0))
        used = final[useable].copy()

        stand_column = used[col] == 0
        used["stand"] = stand_column
        # print("used\n", used.sort_values(cols+[col]))

        # group all games that hit
        grouped = used.groupby(cols + ["stand","initial"]).agg({'chance': 'sum', "expected_return": "sum"}).sort_values("initial").reset_index()
        # print("grouped by cols and stand\n",grouped.sort_values(cols+["stand"]))

        standing_games = used[used["stand"]].copy().drop(columns=["expected_return",col,"stand","stand_return","hit_return"])

        standing_games = pd.merge(standing_games, grouped.loc[grouped["stand"] == True, ["initial","expected_return"]], on=["initial"], how="left")
        standing_games = standing_games.rename(columns={"expected_return":"stand_return"})
        standing_games = pd.merge(standing_games, grouped.loc[grouped["stand"] == False, ["initial","expected_return"]], on=["initial"], how="left")
        standing_games = standing_games.rename(columns={"expected_return":"hit_return"})
        
        standing_games["expected_return"] = np.max(standing_games[["stand_return","hit_return"]], axis=1)
        standing_games["best"] = np.where(standing_games['hit_return'] > standing_games['stand_return'], "hit", "stand")  
        print("standing_games\n", standing_games.sort_values(cols))
        
        # drop drawn column from unused games
        final = final[~useable].copy().drop(columns=[col])

        final = pd.concat([final, standing_games], axis=0, ignore_index=True)

        print(final)

    final = final.drop(columns=["initial"])
    final["normed_return"] = final["expected_return"] / final["chance"]  

    return final

In [199]:
def get_expected_return_from_any(game):
    """get the expected return of the game from any state"""

    if np.all(game["player_cards"] == 0) and np.all(game["dealer_cards"] == 0):
        print("Game is in initial state")
        game = dealer_draws_once(game)
        game = player_draws_once(game)
        game = player_draws_once(game)
        print(game)

    game = get_expected_return_best(game)

    return game

# TESTING

In [210]:
game = get_game(None, [10,9],[7])
game2 = get_game(None, [10,8],[11])
game = pd.concat([game,game2], axis=0, ignore_index=True)
game.loc[1,"chance"] = 0.2
game.loc[0,"chance"] = 0.8
get_expected_return_best(game)

game
    chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
1     0.2            18             2            0            11             1            1  24  24  24  24  24  24  23  24  95  23
0     0.8            19             2            0             7             1            0  24  24  24  24  24  23  24  23  95  24
game
       chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11  initial  drawn_1  drawn_2  drawn_3
0   0.800000            19             2            0             7             1            0  24  24  24  24  24  23  24  23  95  24        0        0        0        0
1   0.200000            18             2            0            11             1            1  24  24  24  24  24  24  23  24  95  23        1        0        0        0
17  0.014887            19             3            0            11            

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return,stand_return,hit_return,best,normed_return
0,0.8,19,2,0,7,1,0,24,24,24,24,24,23,24,23,95,24,0.492038,0.492038,-0.570205,stand,0.615047
1,0.2,18,2,0,11,1,1,24,24,24,24,24,24,23,24,95,23,-0.074962,-0.074962,-0.146862,stand,-0.374811


In [207]:
get_expected_return_from_any(get_game())

Game is in initial state
       chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0    0.000404             4             2            0             2             1            0  21  24  24  24  24  24  24  24  96  24
1    0.000440             4             2            0             3             1            0  22  23  24  24  24  24  24  24  96  24
2    0.000440             4             2            0             4             1            0  22  24  23  24  24  24  24  24  96  24
3    0.000440             4             2            0             5             1            0  22  24  24  23  24  24  24  24  96  24
4    0.000440             4             2            0             6             1            0  22  24  24  24  23  24  24  24  96  24
..        ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
542  0.003677          

KeyError: "['dealer_aces', 'dealer_cards'] not in index"

In [ ]:
# this works well, however the last step is currently not supported
# prints best options, I could just max() the expected_return but this iwll not work for multiple
# 
# next step is to make it work for n input games, I think for that I will need some kind of identifier so that the game knows which initial game to map it to...
# maybe just index it, then map it to the index at the last step (and also the steps before)
 


In [ ]:
pos = player_plays_all_possible(game)
pos[pos["player_score"] > 0].sort_values("player_score")

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,initial,drawn_1,drawn_2,drawn_3
0,0.500000,18,2,0,11,1,1,24,24,24,24,24,24,23,24,95,23,0,0,0,0
1,0.500000,19,2,0,11,1,1,24,24,24,24,24,24,24,23,95,23,1,0,0,0
17,0.037217,19,3,0,11,1,1,24,24,24,24,24,24,23,24,95,22,0,11,0,0
18,0.038835,20,3,0,11,1,1,23,24,24,24,24,24,23,24,95,23,0,2,0,0
19,0.037217,20,3,0,11,1,1,24,24,24,24,24,24,24,23,95,22,1,11,0,0
48,0.002658,20,4,0,11,1,1,24,24,24,24,24,24,23,24,95,21,0,11,11,0
20,0.038835,21,3,0,11,1,1,23,24,24,24,24,24,24,23,95,23,1,2,0,0
21,0.038835,21,3,0,11,1,1,24,23,24,24,24,24,23,24,95,23,0,3,0,0
49,0.002900,21,4,0,11,1,1,23,24,24,24,24,24,23,24,95,22,0,11,2,0
50,0.002900,21,4,0,11,1,1,23,24,24,24,24,24,23,24,95,22,0,2,11,0


In [ ]:
get_expected_return_standing(game)

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return
0,1.0,8,2,0,11,1,1,24,23,24,23,24,24,24,24,96,23,-0.76905


In [ ]:
get_expected_return_best(game)

             chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11  drawn_1  drawn_2  drawn_3  drawn_4  drawn_5  drawn_6  drawn_7  drawn_8  drawn_9  drawn_10  drawn_11  drawn_12  expected_return
0      2.420880e-02            -2             4            0            11             1            1  24  23  23  23  24  24  24  24  95  23        4       10        0        0        0        0        0        0        0         0         0         0    -2.420880e-02
1      9.582650e-02            -2             4            0            11             1            1  24  23  24  23  24  24  24  24  94  23       10       10        0        0        0        0        0        0        0         0         0         0    -9.582650e-02
2      2.420880e-02            -2             4            0            11             1            1  24  23  24  23  24  24  24  23  95  23       10        9        0        0        0    

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,drawn_1,drawn_2,drawn_3,drawn_4,drawn_5,drawn_6,drawn_7,drawn_8,drawn_9,drawn_10,drawn_11,expected_return
0,2.420880e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,23.0,23.0,24.0,24.0,24.0,24.0,95.0,23.0,4,10,0,0,0,0,0,0,0,0,0,-2.420880e-02
1,9.582650e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,24.0,23.0,24.0,24.0,24.0,24.0,94.0,23.0,10,10,0,0,0,0,0,0,0,0,0,-9.582650e-02
2,2.420880e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,24.0,23.0,24.0,24.0,24.0,23.0,95.0,23.0,10,9,0,0,0,0,0,0,0,0,0,-2.420880e-02
3,2.420880e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,24.0,23.0,24.0,24.0,24.0,23.0,95.0,23.0,9,10,0,0,0,0,0,0,0,0,0,-2.420880e-02
4,2.420880e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,24.0,23.0,24.0,24.0,23.0,24.0,95.0,23.0,10,8,0,0,0,0,0,0,0,0,0,-2.420880e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25546,7.992345e-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,2,11,11,11,11,11,2,11,11,2.442108e-14
25547,7.992345e-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,2,11,11,11,11,11,11,2,11,2.442108e-14
25548,7.992345e-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,2,11,11,11,11,11,11,11,2,2.442108e-14
25549,4.864906e-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,2,11,11,11,11,11,11,11,11,6.065802e-15
